In [2]:
import fitz
import easyocr
from PIL import Image
import io
import numpy as np

In [3]:

reader = easyocr.Reader(['pt'], gpu=True)

def extrair_texto_pdf(caminho_pdf: str) -> str:
    texto_total = ""

    # Abre o PDF
    doc = fitz.open(caminho_pdf)

    for pagina in doc:
        texto = pagina.get_text()
        if texto.strip():
            texto_total += texto + "\n"

    if texto_total.strip():
        return texto_total.strip()

    for pagina in doc:
        # Renderiza a página em imagem (matriz de pixels)
        pix = pagina.get_pixmap(dpi=200)
        imagem_bytes = pix.tobytes("png")

        # Converte os bytes em imagem PIL
        imagem_pil = Image.open(io.BytesIO(imagem_bytes)).convert("RGB")

        # Converte PIL para array numpy (formato aceito pelo EasyOCR)
        imagem_array = np.array(imagem_pil)

        # Usa OCR na imagem
        resultado = reader.readtext(imagem_array, detail=0)
        texto_total += "\n".join(resultado) + "\n"

    doc.close()
    return texto_total.strip()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [4]:
import os
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

apiKey = os.environ["open_ai_key"]

llm = ChatOpenAI(model="gpt-4o",api_key=apiKey)

def extrair_dados(texto_da_nota: str) -> str:

    prompt = f"""
    Extraia as seguintes informações do texto abaixo e retorne um JSON com as chaves:
    'prestador', 'cnpj', 'data', 'procedimento', 'valor_total', numero_nota, conselho, tipo_conselho.

    Texto da nota:
    {texto_da_nota}
    """
    # Criar o prompt template
    prompt_template = ChatPromptTemplate.from_template(prompt)

    # Preencher o template com o texto real da nota
    messages = prompt_template.format_messages(texto=texto_da_nota)

    # Chamar o modelo
    return llm(messages)

In [5]:
#texto_extraido = extrair_texto_pdf("NFSe_00003018_06533027 - Fabio.pdf")

#texto_extraido = extrair_texto_pdf("NFSe_00004270_11467997.pdf")

#print(extrair_texto_pdf_sem_poppler("Aposta_250707_125432.pdf"))


resultado = extrair_dados("Nota fiscal referente ao atendimento médico realizado por Dr. João Silva, CRM 123456-SP, CNPJ 12.345.678/0001-90, em 10/07/2025. Procedimento: Consulta. Valor: R$350,00. Número da nota: 20250710001.")

print(resultado)


C:\Users\F_Mota\AppData\Local\Temp\ipykernel_43400\2644713927.py:26: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm(messages)


content='```json\n{\n    "prestador": "Dr. João Silva",\n    "cnpj": "12.345.678/0001-90",\n    "data": "10/07/2025",\n    "procedimento": "Consulta",\n    "valor_total": "R$350,00",\n    "numero_nota": "20250710001",\n    "conselho": "123456-SP",\n    "tipo_conselho": "CRM"\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 130, 'total_tokens': 229, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BxJLaa9lHQDH2v1dQFwvCZGx2zLRk', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--858b5880-50b5-49d9-ac72-f9258f654132-0' usage_metadata={'input_tokens': 130, 'output_tokens': 99, 'total_tokens': 229, 'input_token_details': {'audio': 0, 'cache_read': 

In [6]:
print(resultado.content)

```json
{
    "prestador": "Dr. João Silva",
    "cnpj": "12.345.678/0001-90",
    "data": "10/07/2025",
    "procedimento": "Consulta",
    "valor_total": "R$350,00",
    "numero_nota": "20250710001",
    "conselho": "123456-SP",
    "tipo_conselho": "CRM"
}
```


In [7]:
import xml.etree.ElementTree as ET

def xml_para_texto(xml_path: str) -> str:
    tree = ET.parse(xml_path)
    root = tree.getroot()

    def percorrer(elem, prefix=''):
        linhas = []
        for child in elem:
            tag = child.tag.split('}')[-1]
            valor = child.text.strip() if child.text else ''
            if list(child):
                linhas += percorrer(child, prefix + tag + ".")
            elif valor:
                linhas.append(f"{prefix}{tag}: {valor}")
        return linhas

    return "\n".join(percorrer(root))

In [10]:
texto_extraido = xml_para_texto("NFE_XML.xml")

resultado = extrair_dados(texto_extraido)

print(resultado.content)

```json
{
    "prestador": "Clínica Médica Exemplo",
    "cnpj": "12345678000100",
    "data": "2025-07-22T09:00:00-03:00",
    "procedimento": "Consulta Médica",
    "valor_total": "300.00",
    "numero_nota": "1234",
    "conselho": "004533",
    "tipo_conselho": "CRM"
}
```


In [6]:
import re
import json

def limpar_json_resposta(resposta_llm: str):
    # Remove marcações como ```json e ```
    texto_limpo = re.sub(r"```(?:json)?\n?|```", "", resposta_llm).strip()
    
    try:
        # Primeira tentativa de parsing
        dados = json.loads(texto_limpo)
    except json.JSONDecodeError:
        # Se ainda for uma string JSON escapada, faz o parsing novamente
        dados = json.loads(json.loads(texto_limpo))
    
    return dados

# Exemplo de resposta (com barra invertida e \n)
resposta_llm = '{\n \"prestador\": \"Clínica Médica Exemplo\",\n  \"cnpj\": \"12345678000100\"}'

resultado = limpar_json_resposta(resposta_llm)
print(resultado)

{'prestador': 'Clínica Médica Exemplo', 'cnpj': '12345678000100'}
